## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

%matplotlib inline

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Benjamin Hill,MX,30.1667,-111.1000,80.89,11,33,9.80,scattered clouds
1,1,Shelburne,CA,44.0787,-80.2041,36.75,90,100,5.88,overcast clouds
2,2,Clinton,US,42.5870,-82.9199,43.95,86,75,6.91,broken clouds
3,3,Progreso,MX,21.2833,-89.6667,75.31,51,22,17.56,few clouds
4,4,Saint George,US,37.1041,-113.5841,81.19,12,0,3.44,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation trip? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation trip? "))

What is the minimum temperature you would like for your vacation trip? 50
What is the maximum temperature you would like for your vacation trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
vacation_preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Progreso,MX,21.2833,-89.6667,75.31,51,22,17.56,few clouds
5,5,Kerrville,US,30.0474,-99.1403,63.55,19,0,3.44,clear sky
9,9,Hurricane,US,37.1753,-113.2899,79.27,19,9,1.48,clear sky
10,10,Taos,US,36.4072,-105.5731,55.80,20,0,11.50,clear sky
11,11,Fallon,US,39.4735,-118.7774,73.27,16,75,17.27,broken clouds
15,15,Brandon,US,27.9378,-82.2859,68.11,67,0,5.01,clear sky
18,18,Vidalia,US,32.2177,-82.4135,60.75,39,100,10.36,overcast clouds
25,25,Spearfish,US,44.4908,-103.8594,50.09,39,0,8.05,clear sky
26,26,Groton,US,41.3501,-72.0784,50.76,74,0,3.00,clear sky
28,28,El Reno,US,35.5323,-97.9550,53.33,32,0,5.75,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
vacation_preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Progreso,MX,75.31,few clouds,21.2833,-89.6667,
5,Kerrville,US,63.55,clear sky,30.0474,-99.1403,
9,Hurricane,US,79.27,clear sky,37.1753,-113.2899,
10,Taos,US,55.80,clear sky,36.4072,-105.5731,
11,Fallon,US,73.27,broken clouds,39.4735,-118.7774,
15,Brandon,US,68.11,clear sky,27.9378,-82.2859,
18,Vidalia,US,60.75,overcast clouds,32.2177,-82.4135,
25,Spearfish,US,50.09,clear sky,44.4908,-103.8594,
26,Groton,US,50.76,clear sky,41.3501,-72.0784,
28,El Reno,US,53.33,clear sky,35.5323,-97.9550,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Progreso,MX,75.31,few clouds,21.2833,-89.6667,Playa Linda Hotel
5,Kerrville,US,63.55,clear sky,30.0474,-99.1403,Inn Of The Hills Hotel & Conference Center
9,Hurricane,US,79.27,clear sky,37.1753,-113.2899,Super 8 by Wyndham Hurricane Zion National Park
10,Taos,US,55.80,clear sky,36.4072,-105.5731,El Monte Sagrado
11,Fallon,US,73.27,broken clouds,39.4735,-118.7774,Best Western Fallon Inn & Suites
15,Brandon,US,68.11,clear sky,27.9378,-82.2859,Embassy Suites by Hilton Tampa Brandon
18,Vidalia,US,60.75,overcast clouds,32.2177,-82.4135,Quality Inn & Suites
25,Spearfish,US,50.09,clear sky,44.4908,-103.8594,Travelodge by Wyndham Spearfish
26,Groton,US,50.76,clear sky,41.3501,-72.0784,Red Roof Inn Mystic - New London
28,El Reno,US,53.33,clear sky,35.5323,-97.9550,Home2 Suites by Hilton El Reno


In [10]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(inplace = True)

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))